Literal english meanings of names of Indian states.
![](https://preview.redd.it/vmmbl5uml4151.jpg?width=960&crop=smart&auto=webp&s=d94c0167566189047eee8ae919eca988087c6f04)
https://preview.redd.it/vmmbl5uml4151.jpg?width=960&crop=smart&auto=webp&s=d94c0167566189047eee8ae919eca988087c6f04

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/pandas-bokeh/dataset_tk.csv')
df.head()

#Codes from AmritGrg https://www.kaggle.com/amritgrg/high-accuracy-with-detailed-eda-feature-engineer

In [ ]:
# Numerical features
Numerical_feat = [feature for feature in df.columns if df[feature].dtypes != 'O']
print('Total numerical features: ', len(Numerical_feat))
print('\nNumerical Features: ', Numerical_feat)

In [ ]:
# Zoomed heatmap, correlation matrix
sns.set(rc={'figure.figsize':(12,8)})
correlation_matrix = df.corr()

k = 10             #number of variables for heatmap
cols = correlation_matrix.nlargest(k, 'Rajasthan')['Rajasthan'].index
cm = np.corrcoef(df[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

In [ ]:
## Lets Find the realtionship between discrete features and SalePrice

#plt.figure(figsize=(8,6))

for feature in Numerical_feat:
    data=df.copy()
    plt.figure(figsize=(8,6))
    data.groupby(feature)['Rajasthan'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('Rajasthan')
    plt.title(feature)
    plt.show()

In [ ]:
df[Numerical_feat].hist(bins=25)
plt.show()

In [ ]:
## let us now examine the relationship between continuous features and SalePrice
## Before that lets find continous features that donot contain zero values

continuous_nozero = [feature for feature in Numerical_feat if 0 not in data[feature].unique() and feature not in ['Tripura', 'Nagaland']]

for feature in continuous_nozero:
    plt.figure(figsize=(8,6))
    data = df.copy()
    data[feature] = np.log(data[feature])
    data['Rajasthan'] = np.log(data['Rajasthan'])
    plt.scatter(data[feature], data['Rajasthan'])
    plt.xlabel(feature)
    plt.ylabel('Rajasthan')
    plt.show()

In [ ]:
## Normality and distribution checking for continous features
for feature in continuous_nozero:
    plt.figure(figsize=(6,6))
    data = df.copy()
    sns.distplot(data[feature])
    plt.show()

#Categorical features

In [ ]:
# categorical features
categorical_feat = [feature for feature in df.columns if df[feature].dtypes=='O']
print('Total categorical features: ', len(categorical_feat))
print('\n',categorical_feat)

In [ ]:
# lets find unique values in each categorical features
for feature in categorical_feat:
    print('{} has {} categories. They are:'.format(feature,len(df[feature].unique())))
    print(df[feature].unique())
    print('\n')

In [ ]:
# let us find relationship of categorical with target variable

for feature in categorical_feat:
    data=df.copy()
    data.groupby(feature)['Rajasthan'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('Rajasthan')
    plt.title(feature)
    plt.show()

In [ ]:
# these are selected features from EDA section
features = ['Rajasthan', 'Punjab', 'Uttarakhand', 'UP', 'J&K', 'Meghalaya', 'DNH', 'Unnamed: 0']

In [ ]:
# plot bivariate distribution (above given features with saleprice(target feature))
for feature in features:
    if feature!='Rajasthan':
        plt.scatter(df[feature], df['Rajasthan'])
        plt.xlabel(feature)
        plt.ylabel('Rajasthan')
        plt.show()

#Outliers

We can see a clear otliers in Uttarakhand. I mean it just doesn't make sense for larger values Uttarakhand to have low value of Rajasthan (SalePrice). There might be some reason for this but we'll consider them as outliers and drop them.

In [ ]:
#Deleting outliers for GrLivArea
df = df.drop(df[(df['Uttarakhand']>4000) & (df['Rajasthan']<300000)].index)

plt.scatter(df['Uttarakhand'], df['Rajasthan'])
plt.xlabel('Uttarakhand')
plt.ylabel('Rajasthan')
plt.show()

#Normalizing some numerical data

In [ ]:
# these are selected features from EDA section
features = ['Rajasthan', 'Punjab', 'Uttarakhand', 'UP', 'J&K', 'Meghalaya', 'DNH']

# selecting continuous features from above
continuous_features = ['Rajasthan', 'Uttarakhand']

In [ ]:
#Train = train_df.shape[0]
#Test = test_df.shape[0]
#target_feature = train_df.SalePrice.values
#combined_data = pd.concat((train_df, test_df)).reset_index(drop=True)
#combined_data.drop(['SalePrice','Id'], axis=1, inplace=True)
#print("all_data size is : {}".format(combined_data.shape))

In [ ]:
#Since I have no train, test files, Id, I adapted the code above for just 1 line, so that I could plot the distplot.  
combined_data = pd.concat((df, df)).reset_index(drop=True)

In [ ]:
from scipy.stats import norm

# checking distribution of continuous features(histogram plot)
for feature in continuous_features:
    if feature!='Rajasthan':
        sns.distplot(combined_data[feature], fit=norm)
        plt.show()
    else:
        sns.distplot(df['Rajasthan'], fit=norm)
        plt.show()

#Label encoding, One-Hot-Encoding/dummies

In [ ]:
# so let's label encode above ordinal features
from sklearn.preprocessing import LabelEncoder
for feature in features:
    encoder = LabelEncoder()
    combined_data[feature] = encoder.fit_transform(list(combined_data[feature].values))

In [ ]:
# Now lets see label encoded data
combined_data[features].head()

In [ ]:
## One hot encoding or getting dummies 

dummy_ordinals = pd.get_dummies(features) 
dummy_ordinals.head()

In [ ]:
# creating dummy variables

combined_data = pd.get_dummies(combined_data)
print(combined_data.shape)

In [ ]:
combined_data.head()

#Feature Scaling

In [ ]:
# let's first see descriptive stat info 
combined_data.describe()

#Above, data range differs so much. We need to scale them to same range.

In [ ]:
## we willtake all features from combined_dummy_data 
features_to_scale = [feature for feature in combined_data]
print(len(features_to_scale))

In [ ]:
## Now here is where we will scale our data using sklearn module.

from sklearn.preprocessing import MinMaxScaler

cols = combined_data.columns  # columns of combined_dummy_data

scaler = MinMaxScaler()
combined_data = scaler.fit_transform(combined_data[features_to_scale])

In [ ]:
# after scaling combined_data it is now in ndarray datypes
# so we will create DataFrame from it
combined_scaled_data = pd.DataFrame(combined_data, columns=[cols])

In [ ]:
combined_scaled_data.head() # this is the same combined_dummy_data in scaled form.

In [ ]:
# lets see descriptive stat info 
combined_scaled_data.describe()

Above two dataframe tables that datas are now scaled.

In [ ]:
#That's the code. Though we don't have train nor test, then I adapted once more. 
#train_df.shape, test_df.shape, combined_scaled_data.shape, combined_data.shape

In [ ]:
df.shape, df.shape, combined_scaled_data.shape, combined_data.shape

Initially, train data had 504 observations but I had droped 1 (oo 3 )in outlier handling section so now I have ??? observations.
I simply didn't understand that countability. The original said: "Initially, train data had 1460 observations but we had droped 2 oo 3 in outlier handling section so now we have 4581 observations." And the numbers are: ((1458, 81), (1459, 80), (2919, 225), (2919, 225)). I'll read it later and try to get it.

In [ ]:
# separate train data and test data 
train_data = combined_scaled_data.iloc[:504,:]
test_data = combined_scaled_data.iloc[504:,:]

train_data.shape, test_data.shape

In [ ]:
## lets add target feature to train_data
#train_data['Rajasthan']= train_data['Rajasthan']  # This saleprice is normalized. Its very impportant

In [ ]:
train_data = train_data
train_data.head(10)

In [ ]:
test_data = test_data.reset_index()
test_data.tail()

#Feature Selection

In [ ]:
dataset = train_data.copy()  # copy train_data to dataset variable

In [ ]:
dataset.head()

In [ ]:
dataset = dataset.dropna()

In [ ]:
## lets create dependent and target feature vectors

X = dataset.drop(['Rajasthan'],axis=1)
Y = dataset[['Rajasthan']]

X.shape, Y.shape

In [ ]:
Y.head()

In [ ]:
# lets do feature selection here

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

# define feature selection
fs = SelectKBest(score_func=f_regression, k=27)
# apply feature selection
X_selected = fs.fit_transform(X, Y)
print(X_selected.shape)

We can see that 28 (27+1 outlier?) best/important features have been selected.

In [ ]:
cols = list(range(1,28))

## create dataframe of selected features

selected_feat = pd.DataFrame(data=X_selected,columns=[cols])
selected_feat.head()

In [ ]:
# perform train_test_split
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(selected_feat,Y,test_size=0.3,random_state=0)

In [ ]:
x_train.shape, x_test.shape

#Model Building

LINEAR REGRESSION

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

lr = LinearRegression()
lr.fit(x_train,y_train)

In [ ]:
y_pred = lr.predict(x_test) # predicting test data
y_pred[:10]

In [ ]:
# Evaluating the model
print('R squared score',metrics.r2_score(y_test,y_pred))

print('\nMean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

R square score is prety high almost 80% score which is prety good. MAE, MSE and RMSE values also shows pretty good result. It was written that in AmritGrg's Notebook. I don't know if my result is good or not.

In [ ]:
# check for underfitting and overfitting
print('Train Score: ', lr.score(x_train,y_train))
print('Test Score: ', lr.score(x_test,y_test))

Above train score and test score comparable which is good. Even though it shows a slight case of underfitting but thats fine here. Is it? I don't know.

In [ ]:
## scatter plot of original and predicted target test data
plt.figure(figsize=(8,6))
plt.scatter(y_test,y_pred)
plt.xlabel('y_tes')
plt.ylabel('y_pred')
plt.show()

In [ ]:
## Lets do error plot
## to get error in prediction just substract predicted values from original values

error = list(y_test.values-y_pred)
plt.figure(figsize=(8,6))
sns.distplot(error)

#RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = RandomForestRegressor(n_estimators=100)
rf_reg.fit(x_train,y_train)

In [ ]:
y_pred = rf_reg.predict(x_test)
print(y_pred[:10])

In [ ]:
## evaluating the model

print('R squared error',metrics.r2_score(y_test,y_pred))

print('\nMean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

In [ ]:
# check score
print('Train Score: ', rf_reg.score(x_train,y_train))
print('Test Score: ', rf_reg.score(x_test,y_test))

"We have train set accuracy of 0.9792356047371404 and test set accuracy of 0.8848913083086402. Here we can see overfitting issue but for now we'll leave it alone. They are still pretty good score."  I didn't understand again that words and that numbers.

In [ ]:
## scatter plot of original and predicted target test data
plt.figure(figsize=(8,6))
plt.scatter(y_test,y_pred)
plt.xlabel('y_tes')
plt.ylabel('y_pred')
plt.show()

In [ ]:
## Lets do error plot
## to get error in prediction just substract predicted values from original values

error = list(y_test.values-y_pred)
plt.figure(figsize=(8,6))
sns.distplot(error)

Such a nice error plot? We can see the errors are normally distributed. If you say so.

#Code from Ria https://www.kaggle.com/rai555/google-landmark-retrieval-eda-outlier-analysis/comments

In [ ]:
# Plot
sns.set_style('whitegrid')
fig, (ax1,ax2) = plt.subplots(1, 2)
fig.set_size_inches(18, 6)

sns.countplot(df['Punjab'], order=df['Punjab'].value_counts().index[:20],palette='viridis', ax=ax1)
sns.countplot(df['Rajasthan'], order=df['Rajasthan'].value_counts().index[:20],palette='viridis', ax=ax2)

ax1.tick_params(axis='x', labelrotation=45)
ax2.tick_params(axis='x', labelrotation=45)
ax1.set_title('Punjab')
ax2.set_title('Tripura')
ax2.set(ylim=(0, 100))


plt.show()

![](https://i1.wp.com/gyanshare.in/wp-content/uploads/2019/11/List-of-Indian-states-and-their-languages.jpg?w=800&ssl=1)
https://gyanshare.in/list-of-indian-states-and-their-languages-indian-states-and-their-languages/

Das War's, Kaggle Notebook Runner: Marília Prata  @mpwolke 